---
title: Indonesia GeoMAD Notebook
subtitle: Testing ODC-Stats Configuration for Cloud Cover Optimization and Picking the Right Study and Testing Area
date: 2025-11-13
authors:
  - name: Muhammad Taufik
    affiliations:
      - Badan Informasi Geospasial (BIG)
    email: muhammad.taufik@big.go.id
  - name: Fang Yuan
    affiliations:
      - Auspatious
    email: contact@fangyuan.space
  - name: Alex Leith
    affiliations:
      - Auspatious
    email: alex@auspatious.com

keywords:
  - GeoMAD
  - Sentinel-2
  - Open Data Cube
  - Cloud Cover
  - Indonesia
  - Remote Sensing
  - Earth Observation
project:
  license: CC-BY-4.0
  open_access: true
  github: https://github.com/piksel-ina/Indonesia-geomad
kernelspec:
  name: python3
  display_name: Python 3
---

## Abstract

This notebook explores optimal cloud cover thresholds for generating geoMAD (Geometric Median and Median Absolute Deviation) composites over Indonesia using Sentinel-2 L2A data. We compare different cloud cover filtering strategies (≤100%, ≤80%, ≤60%) to balance data quality and temporal coverage. Additionally, we evaluate suitable study areas for testing and validation across various Indonesia's geographic conditions.

## A. Objectives

1. Evaluate data distribution and availability across Indonesia under different cloud cover thresholds (100%, 80%, 60%)

2. Locate tiles with least datasets to serve as test subjects alongside tiles with diverse geographic conditions

3. Test with Argo Workflows to document peak memory usage, especially on high-dataset tiles


## B. Initial Setup
### Libraries Used
pandas
: Python data analysis library for handling tabular data, DataFrames, and statistical operations. We'll use this to analyze dataset distribution statistics.

odc-stats
: Open Data Cube statistics toolkit for generating temporal composites and summary statistics from Earth observation data. We'll execute this via terminal commands through Jupyter cells.

matplotlib.pyplot
: Python plotting library for creating visualizations. We'll use this to visualize sampling distributions. 

In [ ]:
# import libraries 
import pandas as pd
import matplotlib.pyplot as plt

> In piksel-sandbox, we need to upgrade odc-stats to the latest version.

In [ ]:
!pip install --upgrade odc-stats

## C. Generate ODC-Stats Task Database

> We use terminal commands to imitate the production workflow with odc-stats container.

The function below generates task databases filtered by cloud cover threshold.

In [ ]:
def save_tasks(cloud_cover, output_db):
    !odc-stats save-tasks \
        --frequency "annual" \
        --grid "EPSG:6933;10;5000" \
        --year "2024" \
        --input-products "s2_l2a" \
        --dataset-filter='{{"cloud_cover": [0,{cloud_cover}]}}' \
        {output_db}

### 1. How save_tasks() Works

When executed, the `save-tasks` command performs the following operations:

1. **Query ODC Database** - Connects to the Open Data Cube and queries all indexed Sentinel-2 L2A datasets for the year 2024

2. **Apply Cloud Cover Filter** - The function receives two arguments: `cloud_cover_threshold` and `output_filename`. It filters datasets based on the specified threshold:

   ```python
   save_tasks(60, "tasks_cc60.db")   # cloud_cover: [0, 60]
   save_tasks(80, "tasks_cc80.db")   # cloud_cover: [0, 80]
   save_tasks(100, "tasks_cc100.db") # cloud_cover: [0, 100]
   ```
   
   - **First argument**: Maximum cloud cover percentage (60, 80, or 100)
   - **Second argument**: Output filename prefix for generated files
   - Filters include all datasets with cloud cover from 0% up to the specified threshold

3. **Generate Spatial Grid** - Creates a processing grid in EPSG:6933 projection with 10° tiles at 5000m resolution covering Indonesia

4. **Spatial Intersection** - Matches filtered datasets to their corresponding grid tiles based on spatial footprints

5. **Task Generation** - For each tile, generates processing tasks containing:
   - Tile identifier and spatial bounds
   - List of datasets intersecting that tile
   - Metadata for GeoMAD computation

6. **Database Storage** - Serializes all tasks into multiple output formats:
   - **`.db`** - SQLite database for efficient querying and task distribution
   - **`.csv`** - Tabular summary of tiles and dataset counts
   - **`.json`** - JSON manifest with complete task specifications


In [ ]:
# Execute function (This process takes several minutes to complete)
save_tasks(60, "tasks_cc60.db")
save_tasks(80, "tasks_cc80.db")
save_tasks(100, "tasks_cc100.db")

### 2. Generated Task Files
After executing the save_tasks() function with three different cloud cover thresholds, 
you should have **9 output files** in total, each cloud cover threshold produces **3 files**:

| Cloud Cover | Database | CSV Summary | JSON Manifest |
|-------------|----------|-------------|---------------|
| 0-60% | `tasks_cc60.db` | `tasks_cc60.csv` | `tasks_cc60.json` |
| 0-80% | `tasks_cc80.db` | `tasks_cc80.csv` | `tasks_cc80.json` |
| 0-100% | `tasks_cc100.db` | `tasks_cc100.csv` | `tasks_cc100.json` |

## D. Working with the CSV Files
Now that we have generated the task files, let's analyze the data distribution and availability across Indonesia under different cloud cover thresholds. This analysis will help us understand how cloud filtering impacts dataset availability and identify optimal processing strategies.

### 1. Loading and Inspecting CSV Files
Let's examine the contents of the CSV files using pandas:

In [ ]:
# Load the CSV file for 100% (complete dataset)
df_cc100 = pd.read_csv("tasks_cc100.csv")

> The `pd.read_csv()` command is reading the CSV file and loading it into memory as a pandas DataFrame.

`df_cc100` is a pandas DataFrame object with following rows and columns:

{eval}`df_cc100.head()`

Based on the dataframe output, the CSV files contain the following columns:
- **T** - Time period identifier (e.g., "2024--P1Y" represents year 2024 with 1-year period)
- **X** - Grid tile X-coordinate in the EPSG:6933 projection system
- **Y** - Grid tile Y-coordinate in the EPSG:6933 projection system
- **datasets** - Number of Sentinel-2 datasets intersecting this tile
- **days** - Number of unique observation days available for this tile

### 2. The Descriptive Statistics

Pandas gives us a statistical snapshot of our data; central tendency (mean, median), spread (std, quartiles), and range (min, max). Perfect for understanding the dataset distribution at a glance.

```python
df_cc100.describe()
```
> Here we'll try to extract general information about the dataset `df_cc100`.

**Dataframe Statistic Summary:** \
{eval}`df_cc100.describe()`

:::{important} Total tiles is **{eval}`f"{len(df_cc100):,}"`**

We can also observe some more information about the dataset:  

1. **Datasets per Tile**

    - _Mean_: {eval}`f"{df_cc100['datasets'].mean():.2f}"` datasets per tile
    - _Median_: {eval}`f"{df_cc100['datasets'].median():.0f}"` datasets per tile
    - _Range_: {eval}`f"{df_cc100['datasets'].min():.0f}"` to {eval}`f"{df_cc100['datasets'].max():.0f}"` datasets
    - _Standard deviation_: {eval}`f"{df_cc100['datasets'].std():.2f}"`

2. **Observation Days per Tile**

    - _Mean_: {eval}`f"{df_cc100['days'].mean():.2f}"` days per tile
    - _Median_: {eval}`f"{df_cc100['days'].median():.0f}"` days per tile
    - _Range_: {eval}`f"{df_cc100['days'].min():.0f}"` to {eval}`f"{df_cc100['days'].max():.0f}"` days
    - _Standard deviation_: {eval}`f"{df_cc100['days'].std():.2f}"`
:::



In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

# Create horizontal boxplot
bp = ax.boxplot([df_cc100['datasets'], df_cc100['days']], 
                 vert=False,
                 tick_labels=['Datasets', 'Days'],
                 patch_artist=True)

# Set colors
colors = ['#3498db', '#f39c12']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_xlabel('Count', fontsize=12)
ax.set_ylabel('Metric', fontsize=12)
ax.set_title('Distribution of Datasets and Observation Days per Tile', 
             fontsize=13, fontweight='bold', pad=15)

ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

::::{grid} 2
:::{card}
:header: **What the Data Shows**

Most tiles contain 150-380 datasets spanning 74-146 observation days, with a median of 229 datasets across 145 days. Dataset counts vary significantly more than observation duration, with some tiles receiving multiple satellite passes per day (average 1.6 datasets/day). The spatial distribution is uneven—some areas have substantially more data than others due to satellite orbit patterns.
:::

:::{card}
:header: **Implications for GeoMAD Analysis**

Data-rich tiles will produce more robust median and MAD statistics, while tiles with fewer observations (around 74 days) may not capture full seasonal variations for annual analysis. The spatial imbalance means geoMAD composite quality varies across the region. Areas with limited temporal coverage may be more susceptible to outliers or cloud contamination, affecting the reliability of statistical summaries when comparing results between tiles.
:::
::::


## E. Cloud Cover Filter Analysis

We apply the same analytical method to filtered datasets (`tasks_cc60.csv` and `tasks_cc80.csv`) to evaluate the impact of cloud cover thresholds on data availability and quality.


- **CC60** applies a more aggressive cloud cover filter (≤60% cloud cover allowed)  
- **CC80** applies a more lenient filter (≤80% cloud cover allowed)


### 1. The Question

**How do we balance output quality with dataset availability?**

Larger cloud cover filters reduce the number of cloudy datasets, which should improve output image quality. However, we must maintain a minimum dataset count to provide sufficient information for robust statistical analysis, particularly for **Median Absolute Deviation (MAD)** calculations.


### 2. Comparative Analysis

We compare two configurations (60% and 80% cloud cover thresholds) across two dimensions:

::::{grid} 2
:gutter: 3

:::{grid-item-card} Dataset Count Analysis
:class-header: bg-light

- Number of available datasets per tile
- Temporal coverage consistency
- Spatial distribution patterns
- Data retention rates vs. baseline (CC100)

:::

:::{grid-item-card} Output Quality Assessment
:class-header: bg-light

- Image clarity and cloud contamination
- Statistical robustness (MAD reliability)
- Spatial completeness
- Temporal representativeness

:::

::::


### 3. Expected Trade-offs

```{list-table}
:header-rows: 1
:name: tradeoff-table

* - Metric
  - CC60 (Aggressive)
  - CC80 (Lenient)
* - Dataset Count
  - Lower
  - Higher
* - Cloud Contamination
  - Minimal
  - Moderate
* - Statistical Power
  - Risk of insufficient data
  - Better temporal sampling
* - Output Quality
  - Cleaner imagery
  - Potential artifacts



In [ ]:
# Load the other CSV files 
df_cc80 = pd.read_csv("tasks_cc80.csv")
df_cc60 = pd.read_csv("tasks_cc60.csv")

### 4. Data Availibility and Distribution
:::: {grid} 2
::: {grid-item}
```{code}python
:label: descriptive-analysis-cloud-cover-80
:caption: descriptive statistics of filtered dataset with cloud-cover 0 to 80%
df_cc80.describe()
```


{eval}`df_cc80.describe()`
:::

::: {grid-item}
```{code}python
:label: descriptive-analysis-cloud-cover-60
:caption: descriptive statistics of filtered dataset with cloud-cover 0 to 60%
df_cc60.describe()
```


{eval}`df_cc60.describe()`
:::



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'wspace': 0.5})

colors = ['#3498db', '#f39c12']

# Column 1: Datasets comparison
bp1 = ax1.boxplot([df_cc80['datasets'], df_cc60['datasets']], 
                   vert=False,
                   tick_labels=['80CC', '60CC'],
                   patch_artist=True)

for patch, color in zip(bp1['boxes'], colors):
    patch.set_facecolor(color)

ax1.set_xlabel('Dataset Count per Tile', fontsize=12)
ax1.set_ylabel('CC Threshold', fontsize=12)
ax1.grid(True, alpha=0.3, axis='x')

# Column 2: Days comparison
bp2 = ax2.boxplot([df_cc80['days'], df_cc60['days']], 
                   vert=False,
                   tick_labels=['80CC', '60CC'],
                   patch_artist=True)

for patch, color in zip(bp2['boxes'], colors):
    patch.set_facecolor(color)

ax2.set_xlabel('Observation Days Count per Tile', fontsize=12)
ax2.set_ylabel('CC Threshold', fontsize=12)
ax2.grid(True, alpha=0.3, axis='x')

plt.show()

### 5. Cloud Cover Threshold Analysis

:::{card}
:header: **Overview**
- We analyzed three cloud cover thresholds (100%, 80%, 60%) to determine optimal filtering for our satellite imagery dataset.
- The analysis reveals that **80% threshold offers the best balance**, though **60% remains acceptable**.
:::

::::{grid} 1 1 3 3
:gutter: 3

:::{card} Data Coverage Impact
**Location Retention:**
- **100%**: 3,755 locations
- **80%**: 3,749 locations (99.8%)
- **60%**: 3,749 locations (99.8%)

**Key Insight:** Loss of only 6 locations indicates nearly all grid points have sufficient usable data, even with stricter cloud filtering.
:::

:::{card} Dataset Availability
**Mean / Median Datasets:**
- **100%**: 277 / 229
- **80%**: 162 / 140 (-41.5%)
- **60%**: 121 / 103 (-56.3%)

**Key Insight:** 80% removes ~42% of datasets while maintaining 140 median images per location. 60% removes over half the data but still provides 103 median images - still sufficient for analysis.
:::

:::{card} Temporal Coverage
**Mean / Median Days:**
- **100%**: 124 / 145 days
- **80%**: 81 / 86 days (-34.7%)
- **60%**: 64 / 64 days (-48.5%)

**Key Insight:** 80% provides 86 median days - sufficient for seasonal analysis. 60% reduces to 64 days (~2 months coverage) which may create temporal gaps but ensures each observation is high-quality.
:::

::::

:::{card} 
:header: **Next Steps**
To make a final decision, we need to **visually compare outputs** from different thresholds by sampling representative tiles based on dataset and day count distributions.
:::

## F. Testing Strategy

### 1. Sampling Strategy

To finalize the threshold decision, we need to **visually compare outputs** by sampling representative tiles.

::::{grid} 1 1 2 2
:gutter: 3

:::{card} 
:header: **Sampling Dimensions**
1. Dataset Count Bins:
    - Low: < 25th percentile
    - Medium: 25th-75th percentile  
    - High: > 75th percentile

2. Day Count Bins:
    - Low: < 25th percentile
    - Medium: 25th-75th percentile
    - High: > 75th percentile
:::

:::{card} 
:header: **Test Distribution**

Create a **3×3 sampling matrix**:

| Days \ Datasets | Low | Med | High |
|----------------|-----|-----|------|
| **Low** | 2 tiles | 2 tiles | 2 tiles |
| **Medium** | 2 tiles | 3 tiles | 2 tiles |
| **High** | 2 tiles | 2 tiles | 2 tiles |

- Total Tiles: **19 tiles**
- Total Processing:
    - 19 tiles at 80% & 60% = 38 images
    - 5 tiles additionally at 100% = 5 images
    - Total output: **43 images**
:::

::::

::::{grid} 1 1 2 2
:gutter: 3